In [1]:
import gensim as gs
import os
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
#This csv contains several irritating duplicates
wordChain=pd.read_csv("./wordChains.csv")
wc={}
count=0
for i in wordChain.index:
    w=wordChain.loc[i]["Word"].lower()
    if w not in wc.keys():
        wc[w]=wordChain.loc[i]
        count+=1
wordChain=pd.DataFrame(wc).transpose()[["PoS","Theme","Category"]]
G=nx.DiGraph()

In [3]:
# vocab may not include all words in the csv since there may be some for which no word vectors exist
vocab=wordChain.index

In [4]:
for w1 in vocab:
    for w2 in vocab:
        if(w1[-1]==w2[0]):
            G.add_edge(w1.lower(),w2.lower())

In [5]:
chain_list=[]
for z in range(1,100):
    ls=[wordChain.index[np.random.randint(len(wordChain))]]
    key_list=["PoS","Theme","Category"]
    total_score=0
    for i in range(1,5):
        max_score=0
        w_next=""
        temp=G[ls[-1]]
        for j in temp:
            if(j in ls):
                continue
            obj1=wordChain.loc[ls[-1]]
            obj2=wordChain.loc[j]
            score=0
            for key in key_list:
                if(obj1[key]==obj2[key]):
                    score+=1
            if(score>max_score):
                w_next=j
                max_score=score
        if(w_next!=""):
            ls.append(w_next)
            total_score+=max_score
    if(total_score>10 and (ls,total_score) not in chain_list):
        chain_list.append((ls,total_score))
chain_list

[(['necklace', 'earring', 'garland', 'dress', 'socks'], 12),
 (['nails', 'stomach', 'hair', 'room', 'mirror'], 11),
 (['belt', 'turban', 'necklace', 'earring', 'garland'], 12),
 (['socks', 'slipper', 'ribbon', 'necklace', 'earring'], 12),
 (['turban', 'necklace', 'earring', 'garland', 'dress'], 12),
 (['rat', 'turtle', 'elephant', 'tiger', 'rabbit'], 12),
 (['air', 'rain', 'nails', 'stomach', 'hair'], 11),
 (['mirror', 'room', 'mug', 'glass', 'soap'], 12),
 (['turnip', 'pumpkin', 'nails', 'stomach', 'hair'], 11),
 (['turtle', 'elephant', 'tiger', 'rat', 'tap'], 11),
 (['roof', 'floor', 'room', 'mirror', 'radish'], 11),
 (['ant', 'turtle', 'elephant', 'tiger', 'rat'], 11),
 (['eraser', 'ruler', 'room', 'mirror', 'roof'], 11),
 (['floor', 'room', 'mirror', 'roof', 'fan'], 12),
 (['dress', 'socks', 'slipper', 'ribbon', 'necklace'], 12),
 (['bedroom', 'mirror', 'room', 'mug', 'glass'], 12),
 (['chair', 'room', 'mirror', 'roof', 'floor'], 11),
 (['slipper', 'ribbon', 'necklace', 'earring', 

In [6]:
word_vectors={}
f=open("word_vectors.txt","r")
for line in f.readlines():
    word= line.split(" ")[0]
    vector= line.split(" ")[1]
    if(word.lower() in vocab):
        word_vectors[word]=np.array([float(i) for i in vector.split(",")])
f.close()
len(word_vectors)

450

In [7]:
for key in word_vectors.keys():
    word_vectors[key]/=np.linalg.norm(word_vectors[key])

In [8]:
#New vocab is subset of original vocab
sem_vocab=word_vectors.keys()
sem_G=G.subgraph(sem_vocab)

In [9]:
def similarity(v1,v2):#Assuming normalised vectors (which has already been done above)
    return (np.dot(np.transpose(v1),v2))

In [10]:
chain_list=[]
threshold_score=2
for z in range(1,30):
    ls=[sem_vocab[np.random.randint(len(sem_vocab))]]
    total_score=0
    for i in range(1,5):
        max_score=0
        w_next=""
        temp=sem_G[ls[-1]]
        for j in temp:
            score=0
            if(j in ls):
                continue
            for word_ch in ls:
                score+=similarity(word_vectors[word_ch],word_vectors[j])
            if(score>max_score):
                w_next=j
                max_score=score
        if(w_next!=""):
            ls.append(w_next)
            total_score+=max_score
    if(total_score>threshold_score and (ls,total_score) not in chain_list):
        chain_list.append((ls,total_score))
chain_list

[(['lady', 'yellow', 'white', 'eraser', 'red'], 4.431834770083654),
 (['carry', 'yellow', 'white', 'eraser', 'red'], 4.1460875275599403),
 (['pumpkin', 'necklace', 'earring', 'guava', 'apple'], 5.4834985467841859),
 (['like', 'eat', 'think', 'know', 'want'], 5.2098665621137688),
 (['about', 'then', 'now', 'when', 'not'], 4.2134172556632006),
 (['know', 'want', 'think', 'knew', 'worry'], 5.686711951204579),
 (['out', 'together', 'room', 'mug', 'garden'], 3.1502800131930946),
 (['five', 'eight', 'three', 'eraser', 'radish'], 4.9243743980554306),
 (['cat', 'turtle', 'elephant', 'tiger', 'rabbit'], 6.3343513831439218),
 (['small', 'little', 'elephant', 'turtle', 'eagle'], 3.8950744532175401),
 (['cow', 'watermelon', 'necklace', 'earring', 'guava'], 5.7617726243921918),
 (['read', 'done', 'eat', 'take', 'excited'], 4.0489615198917432),
 (['toes', 'stomach', 'hair', 'radish', 'hat'], 4.784298372068851),
 (['dry', 'yellow', 'wet', 'thin', 'new'], 4.4106132078707905),
 (['chair', 'room', 'mug'